<a href="https://colab.research.google.com/github/Weizhuo-Zhang/ML_coursera/blob/master/hands_on_ML/7_ensemble_learning_and_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 七、集成学习和随机森林 (Ensemble Learning and Ransom Forest)

## 投票分类



In [25]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf),
    ('svc', svm_clf)], voting='hard')
voting_clf.fit(X_train, y_train);

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [28]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.85
RandomForestClassifier 0.95
SVC 0.9
VotingClassifier 0.9


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", Futur

## Bagging 和 Pasting

就像之前讲到的，可以通过使用不同的训练算法去得到一些不同的分类器。另一种方法就是对每一个分类器都使用相同的训练算法，但是在不同的训练集上去训练它们。有放回采样被称为装袋（`Bagging`，是 `bootstrap aggregating` 的缩写）。无放回采样称为粘贴（`pasting`）。

换句话说，`Bagging` 和 `Pasting` 都允许在多个分类器上对训练集进行多次采样，但只有 `Bagging` 允许对同一种分类器上对训练集进行进行多次采样。采样和训练过程如图7-4所示。

### Sklearn 中的 Bagging 和 Pasting

`Sklearn` 为 `Bagging` 和 `Pasting` 提供了一个简单的API： `BaggingClassifier` 类（或者对于回归可以是 `BaggingRegressor` 。接下来的代码训练了一个 500 个决策树分类器的集成，每一个都是在数据集上有放回采样 100 个训练实例下进行训练（这是 `Bagging` 的例子，如果你想尝试 `Pasting`，就设置 `bootstrap=False` ）。 `n_jobs` 参数告诉 `Sklearn` 用于训练和预测所需要 `CPU`核的数量。（-1 代表着 sklearn 会使用所有空闲核）：

In [0]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
    max_samples=80, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_train)

如果基分类器可以预测类别概率（例如它拥有 `predict_proba()` 方法），那么 `BaggingClassifier` 会自动的运行软投票，这是决策树分类器的情况。

### Out-of-Bag 评价

对于 `Bagging` 来说，一些实例可能被一些分类器重复采样，但其他的有可能不会被采样。 `BaggingClassifier` 默认采样。 `BaggingClassifier` 默认是有放回的采样 m 个实例（ `bootstrap=True` ），其中 m 是训练集的大小，这意味着平均下来只有63%的训练实例被每个分类器采样，剩下的37%个没有被采样的训练实例就叫做 `Out-of-Bag` 实例。注意对于每一
个的分类器它们的 37% 不是相同的。

因为在训练中分类器从来没有看到过 `Out-of-Bag` 实例，所以它可以在这些实例上进行评估，而不需要单独的验证集或交叉验证。你可以拿出每一个分类器的 oob 来评估集成本身。

在 `sklearn` 中，你可以在训练后需要创建一个 `BaggingClassifier` 来自动评估时设置 `oob_score=True` 来自动评估。接下来的代码展示了这个操作。评估结果通过变
量 `oob_score_` 来显示：

In [37]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, bootstrap=True,
    n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.925

In [38]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

对于每个训练实例 oob 决策函数也可通过 `oob_decision_function_` 变量来展示。在这种情况下（当基决策器有 `predict_proba()` 时）决策函数会对每个训练实例返回类别概率。例如，oob 评估预测第二个训练实例有 60.6% 的概率属于正类（39.4% 属于负类）：

In [44]:
bag_clf.oob_decision_function_

array([[0.02538071, 0.97461929],
       [0.16176471, 0.83823529],
       [0.24731183, 0.75268817],
       [0.01052632, 0.98947368],
       [0.        , 1.        ],
       [0.12234043, 0.87765957],
       [0.97222222, 0.02777778],
       [0.        , 1.        ],
       [0.59649123, 0.40350877],
       [0.99453552, 0.00546448],
       [0.97382199, 0.02617801],
       [0.84803922, 0.15196078],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.01621622, 0.98378378],
       [0.93717277, 0.06282723],
       [1.        , 0.        ],
       [0.73469388, 0.26530612],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.05780347, 0.94219653],
       [1.        , 0.        ],
       [0.29281768, 0.70718232],
       [0.14659686, 0.85340314],
       [0.24858757, 0.75141243],
       [0.04891304, 0.95108696],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.

## 随机贴片与随机子空间

`BaggingClassifier` 也支持采样特征。它被两个超参数 `max_features` 和 `bootstrap_features` 控制。他们的工作方式和 `max_samples` 和 `bootstrap` 一样，但这是对于特征采样而不是实例采样。因此，每一个分类器都会被在随机的输入特征内进行训练。

当你在处理高维度输入下（例如图片）此方法尤其有效。对训练实例和特征的采样被叫做随机贴片。保留了所有的训练实例（例如 `bootstrap=False` 和 `max_samples=1.0` ），但是对特征采样（ `bootstrap_features=True` 并且/或者 `max_features` 小于 1.0）叫做随机子空间。采样特征导致更多的预测多样性，用高偏差换低方差。

## 随机森林

正如我们所讨论的，随机森林是决策树的一种集成，通常是通过 `bagging` 方法（有时是 `pasting` 方法）进行训练，通常用 `max_samples` 设置为训练集的大小。与建立一个 `BaggingClassifier` 然后把它放入 `DecisionTreeClassifier` 相反，你可以使用更方便的也是对决策树优化够的 `RandomForestClassifier` （对于回归是 `RandomForestRegressor` ）。接下来
的代码训练了带有 500 个树（每个被限制为 16 叶子结点）的决策森林，使用所有空闲的CPU 核：

In [0]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

除了一些例外， `RandomForestClassifier` 使用 `DecisionTreeClassifier` 的所有超参数（决定数怎么生长），把 `BaggingClassifier` 的超参数加起来来控制集成本身。


随机森林算法在树生长时引入了额外的随机；与在节点分裂时需要找到最好分裂特征相反（详见第六章），它在一个随机的特征集中找最好的特征。它导致了树的差异性，并且再一次用高偏差换低方差，总的来说是一个更好的模型。以下是 `BaggingClassifier` 大致相当于之前的 `randomForestClassifier` ：

In [0]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### 极端随机树

### 特征重要度

最后，如果你观察一个单一决策树，重要的特征会出现在更靠近根部的位置，而不重要的特征会经常出现在靠近叶子的位置。因此我们可以通过计算一个特征在森林的全部树中出现的平均深度来预测特征的重要性。`Sklearn` 在训练后会自动计算每个特征的重要度。你可以通过 `feature_importances_` 变量来查看结果。例如如下代码在 iris 数据集（第四章介绍）上训练了一个 `RandomForestClassifier` 模型，然后输出了每个特征的重要性。看来，最重要的特征是花瓣长度（44%）和宽度（42%），而萼片长度和宽度相对比较是不重要的（分别为 11% 和 2%）：



In [55]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09398536614729343
sepal width (cm) 0.025942194025263703
petal length (cm) 0.449178296090451
petal width (cm) 0.4308941437369918


## 提升 (Boosting)

提升（`Boosting`，最初称为假设增强）指的是可以将几个弱学习者组合成强学习者的集成方法。对于大多数的提升方法的思想就是按顺序去训练分类器，每一个都要尝试修正前面的分类。现如今已经有很多的提升方法了，但最著名的就是 `Adaboost`（适应性提升，是 `Adaptive Boosting` 的简称） 和 `Gradient Boosting`（梯度提升）。

### Adaboost

使一个新的分类器去修正之前分类结果的方法就是对之前分类结果不对的训练实例多加关注。这导致新的预测因子越来越多地聚焦于这种情况。这是 Adaboost 使用的技术。

举个例子，去构建一个 Adaboost 分类器，第一个基分类器（例如一个决策树）被训练然后在训练集上做预测，在误分类训练实例上的权重就增加了。第二个分类机使用更新过的权重然后再一次训练，权重更新，以此类推（详见图 7-7）

sklearn 通常使用 Adaboost 的多分类版本 SAMME（这就代表了 分段加建模使用多类指数损失函数）。如果只有两类别，那么 SAMME 是与 Adaboost 相同的。如果分类器可以预测类别概率（例如如果它们有 predict_proba() ），如果 sklearn 可以使用 SAMME 叫做 SAMME.R 的变量（R 代表“REAL”），这种依赖于类别概率的通常比依赖于分类器的更好。

In [56]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
    n_estimators=200, algorithm="SAMME.R", learning_rate=0.5)
print(ada_clf.fit(X_train, y_train))

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

### Gradient Boost 梯度提升

In [0]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

# 现在在第一个分类器的残差上训练第二个分类器
y2 = y_train - tree_reg1.predict(X_train)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train, y2)

# 现在在第二个分类器的残差上训练第三个分类器
y3 = y2 - tree_reg2.predict(X_train)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X_train, y3)

y_pred = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))

我们可以使用 sklean 中的 `GradientBoostingRegressor` 来训练 GBRT (Gradient Tree Boosting or Gradient Boosted Regression Tree) 集成。与 `RandomForestClassifier` 相似，它也有超参数去控制决策树的生长（例如 `max_depth` ， `min_samples_leaf` 等等），也有超参数去控制集成训练，例如基分类器的数量（ `n_estimators` ）。接下来的代码创建了与之前相同的集成：

In [60]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

超参数 `learning_rate` 确立了每个树的贡献。如果你把它设置为一个很小的树，例如 0.1，在集成中就需要更多的树去拟合训练集，但预测通常会更好。这个正则化技术叫做 `shrinkage`。

为了找到树的最优数量，你可以使用早停技术（第四章讨论）。最简单使用这个技术的方法就是使用 `staged_predict()`` ：它在训练的每个阶段（用一棵树，两棵树等）返回一个迭代器。加下来的代码用 120 个树训练了一个 GBRT 集成，然后在训练的每个阶段验证错误以找到树的最佳数量，最后使用 GBRT 树的最优数量训练另一个集成：

In [61]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)
errors = [mean_squared_error(y_val, y_pred)
    for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=85,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

你可以通过设置 `warm_start=True` 来实现 ，这使得当 `fit()` 方法被调用时 `sklearn` 保留现有树，并允许增量训练。接下来的代码在当一行中的五次迭代验证错误没有改善时会停止
训练：

In [0]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)
min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break

## Stacking

**Stacking** (Stacked generalization) 

这个算法基于一个简单的想法：不使用琐碎的函数（如硬投票）来聚合集合中所有分类器的预测，我们为什么不训练一个模型来执行这个聚合？图 7-12 展示了这样一个在新的回归实例上预测的集
成。底部三个分类器每一个都有不同的值（3.1，2.7 和 2.9），然后最后一个分类器（叫做 `blender` 或者 `meta learner` ）把这三个分类器的结果当做输入然后做出最终决策（3.0）。

![stackingGenerization](https://raw.githubusercontent.com/Weizhuo-Zhang/ML_coursera/master/hands_on_ML/pics/stackingGenerization.jpg)